In [25]:
import cv2
import os
import sys
from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F
from IPython.display import clear_output, Video
from PIL import Image

cap = cv2.VideoCapture("./observations_gripper_rgb.mp4")
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('./clip_short_cosine.mp4',fourcc, 30.0, (720,544))

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

annotations = ["onion", "basketball", "egg yolk", "iron (for clothes)"]
annotations_long = [
    "Onion: Layered bulb with papery skin, transitioning from pale white to earthy brown. Circular cross-section.",
    "Basketball: A spherical orange object with black seams, designed for dribbling, shooting, and passing in basketball.",
    "Egg Yolk: A golden yolk encircled by a glistening, opaque white, basking in morning sunlight's warm embrace.",
    "Iron (for clothes): Smooth, sleek, flat device with a pointed tip, used to remove wrinkles from clothes."
]

while(cap.isOpened()):
    ret, im = cap.read()
    if ret:
        converted = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
        pil_im = Image.fromarray(converted)
        
        inputs = processor(text=annotations, images=pil_im, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        
        # logits_per_image = outputs.logits_per_image # this is the image-text similarity score
        # probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
        
        text_embeds = outputs.text_embeds
        image_embeds = outputs.image_embeds
        cosine = F.cosine_similarity(text_embeds, image_embeds)
        
        for i in range(len(annotations)):
            cv2.putText(im, annotations[i] + ": " + str(round(cosine[i].item(), 3)), (10, 40 + 35*i), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        out.write(im)
    else:
        break

cap.release()
out.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
